## Moloco Data Scientist, Ad Operations Screening exercise Part-1

NAME: Kunal Teeda

Mail ID: kunal.teeda@gmail.com


In [3]:
#Importing the required libraries 
import pandas as pd             # manipulting dataframes 
import numpy as np              # manipulating arrays, scientific computations
import datetime as dt

In [5]:
# loading the csv file into a dataframe
df = pd.read_csv('Adops & Data Scientist Sample Data - Q1 Analytics.csv')

### Initial EDA

In [6]:
#printing out the first few rows in our dataframe
df.head(5)

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [7]:
#Printing shape of data
df.shape

(3553, 4)

In [8]:
#checking the datatypes of elements in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
ts            3553 non-null object
user_id       3553 non-null object
country_id    3553 non-null object
site_id       3553 non-null object
dtypes: object(4)
memory usage: 111.1+ KB


### Part-1

A. Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [9]:
#filter by country id equal BDV
df_bdv = df[df.country_id == 'BDV'] 
#group by site id and counting unique user_id
df_bdv_group = df_bdv.groupby('site_id')['user_id'].nunique().to_frame() 

# sort by unique user number and returning the largest record
df_bdv_group.sort_values('user_id',ascending = False).head(1)

,user_id
site_id,
5NPAU,544


B. Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [10]:
#convert ts to timestamp
df.ts = pd.to_datetime(df.ts) 
begin_t = dt.datetime(2019,2,3,0,0,0) #2019-02-03 00:00:00
end_t = dt.datetime(2019,2,4,23,59,59) #2019-02-04 23:59:59

#filter by time stamp
df_time = df.loc[(df.ts>begin_t)&(df.ts < end_t)] 

#visit count for each user
df_count = df_time.groupby(['user_id','site_id'])['ts'].count().to_frame().reset_index()
#visit times greater than 10
df_count[df_count.ts > 10] 

,user_id,site_id,ts
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


C. For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [11]:
#return the latest visit time
df_last_visit = df.groupby('user_id')['ts'].max().to_frame().reset_index()
#left join previous table with the entire table to get last visit site
df_last_site = df_last_visit.merge(df, on = ['ts','user_id'], how='left' )
#renaming columns
df_last_site.columns = ['user_id','ts_last', 'country_last', 'site_last']
#group by last vist site and count site occurrences
df_last_site.groupby('site_last')['ts_last'].count().to_frame().sort_values('ts_last', ascending = False)

,ts_last
site_last,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28
RT9Z6,2
EUZ/Q,1
JSUUP,1


D. For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [12]:
#return first visit
df_first_visit = df.groupby('user_id')['ts'].min().to_frame().reset_index()
#left join previous table with the entire table to get first visit site
df_first_site = df_first_visit.merge(df, on = ['ts','user_id'], how='left' )
#renaming columns
df_first_site.columns = ['user_id','ts_first', 'country_first', 'site_first']
#combine with the last visit table we get from previous question, based on user_id
df_comb = df_last_site.merge(df_first_site, on ='user_id', how = 'left')
#filter by last visit site equal first visit site and compute the length
len(df_comb[df_comb.site_first == df_comb.site_last])

1670